In [7]:
!pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks

In [9]:
pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks


Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os

load_dotenv() 

True

In [2]:
os.environ['PINECONE_API_KEY'] =os.getenv('PINECONE_API_KEY')

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
# create the index
# os.environ['PINECONE_API_KEY'] =os.getenv('PINECONE_API_KEY')
from pinecone import Pinecone ,ServerlessSpec
index_name ='hybird-search-langchain-pinecone'

#initializze the Pinecone client

pc = Pinecone()

#ctreat the index

if index_name not in pc.list_indexes().names():
    pc.create_index(
            name = index_name ,
            dimension= 384 ,# dimension of dense vector
            metric= 'dotproduct', # sparse values supported only for dotproduct
            spec = ServerlessSpec(cloud ='aws', region ='us-east-1')


    )

d:\GENAI\SearchEngine\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
index = pc.Index(index_name)
index

In [6]:
## vector embedding and sparse matrix
os.environ['HF_TOKEN'] =os.getenv('HF_TOKEN')



from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
# sparse  matrix

from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [ ]:
sentences = [
        "In 2023 i visited Paris",
        "In 2024 , I visited New york",
        "in 2025 ,i visited Dubai"


]


# tfidf values on these sentence
bm25_encoder.fit(sentences)

## stores the values to a json file 
bm25_encoder.dump('bm25_values.json')

100%|██████████| 3/3 [00:00<00:00, 2243.34it/s]


In [8]:
retriever = PineconeHybridSearchRetriever(embeddings= embeddings , sparse_encoder= bm25_encoder , index = index)

retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002BE40C5FA60>, index=<pinecone.data.index.Index object at 0x000002BE14B9C7C0>)

In [9]:
retriever.add_texts (
     [
        "In 2023 i visited Paris",
        "In 2024 , I visited New york",
        "in 2025 ,i visited Dubai"


]
)

100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


In [10]:
retriever.invoke ('what city did i visit last ')

[Document(metadata={'score': 0.335044444}, page_content='In 2024 , I visited New york'),
 Document(metadata={'score': 0.306964606}, page_content='In 2023 i visited Paris'),
 Document(metadata={'score': 0.262760609}, page_content='in 2025 ,i visited Dubai')]

In [11]:
retriever.invoke ('what city did i visit recent ')

[Document(metadata={'score': 0.303760499}, page_content='In 2024 , I visited New york'),
 Document(metadata={'score': 0.268898427}, page_content='In 2023 i visited Paris'),
 Document(metadata={'score': 0.231094927}, page_content='in 2025 ,i visited Dubai')]